In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import cv2

from topostats.io import LoadScans
from topostats.filters import Filters

from skimage.morphology import label
from skimage.measure import regionprops
from skimage.filters.rank import (
    entropy,
    equalize,
    gradient,
    gradient_percentile,
    maximum,
    mean,
    mean_bilateral,
    mean_percentile,
    median,
    minimum,
    modal,
    noise_filter,
    percentile,
    otsu,
    sum_bilateral,
    threshold,
    threshold_percentile,
    windowed_histogram,
)
from skimage.filters import gaussian, threshold_local, frangi, gabor, sobel, hessian, laplace
from skimage.feature import hessian_matrix, hessian_matrix_eigvals, canny
from skimage.segmentation import active_contour
from scipy.ndimage import (
    distance_transform_edt,
    distance_transform_cdt,
    distance_transform_bf,
    distance_transform_edt,
    distance_transform_bf,
    convolve,
    generic_filter,
)

In [ ]:
def plot(image: np.ndarray, title="", figsize=(8, 8), display_range=False):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.imshow(image)
    if display_range:
        title = title + f" range: {np.min(image), np.max(image)}"
    ax.set_title(title)
    plt.show()


def plot_gallery(images: list, figsize=(20, 40)):
    cols = 5
    rows = int(np.floor(len(images) / cols) + 1)
    print(f"images: {len(images)} rows: {rows}, cols: {cols}")
    fig, ax = plt.subplots(int(rows), int(cols), figsize=figsize)
    for index, image in enumerate(images):
        ax[int(np.floor(index / cols)), int(index % cols)].imshow(image)
    plt.show()


def detect_ridges(gray, sigma=1.0):
    H_elems = hessian_matrix(gray, sigma=sigma, order="rc")
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    return maxima_ridges, minima_ridges

In [ ]:
def get_neighbours(point, shape):
    y, x = point
    neighbours = [
        (ny, nx)
        for ny, nx in [(y - 1, x), (y + 1, x), (y, x - 1), (y, x + 1)]
        if 0 <= ny < shape[0] and 0 <= nx < shape[1]
    ]
    return neighbours


def calc_zeros_in_range(array: np.ndarray, range: int):
    kernel = np.ones((range, range))

    array_int = array.astype(int)

    array_inverted = 1 - array_int

    result = convolve(array_inverted, kernel, mode="constant", cval=0.0)

    return result


def filter_by_percentile(arr, percentile: float, kernel_size=3):
    # Define the function to apply to each window
    index_of_middle = np.square(kernel_size) // 2

    def func(window):
        # Find the median value of the window
        threshold = np.percentile(window, percentile)
        # Set the center pixel to 0 if it is less than the median
        if window[index_of_middle] < threshold:
            return 0
        else:
            return window[index_of_middle]

    # Create a new array by applying the function to each 3x3 window
    result = generic_filter(arr, func, size=kernel_size, mode="constant", cval=0.0)

    return result

### Load images

In [ ]:
FILE_DIR = Path("/Users/sylvi/topo_data/cats/flattened_images_numpy/")
# OUTPUT_DIR = Path("/Users/sylvi/topo_data/cats/flattened_images_numpy/")
CONFIG_DIR = Path("/Users/sylvi/topo_data/cats/catsconf.yaml")
files = FILE_DIR.glob("*.npy")

# images = []
# for file in files:
# print(file)
# image = cv2.imread(str(file), 0)
# images.append(image)
# plot(image)

images = []
for file in files:
    image = np.load(file)
    images.append(image)


threshold = 1.8
masks = []
for image in images:
    mask = image > threshold
    masks.append(mask)

In [ ]:
# Start at generous threshold

# Threshold

# Label regions

# Count holes

# Lower threshold

# Label regions

# Count holes

# If holes number has increased check hole size

# If hole size is small, ban that hole

image = images[6]
thresholded_images = []
for threshold in reversed(np.arange(1, 3, 0.2)):
    thresholded_images.append(image > threshold)

plot(image)
for thresholded_image in thresholded_images:
    plot(thresholded_image)

### Experiment with filters

In [ ]:
from skimage.morphology import disk
from scipy.ndimage import laplace as scipy_laplace
import cv2
import numpy as np
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import data, img_as_float


image = images[3].copy()

image = gaussian(image, sigma=1)

plot(image)

minima, maxima = detect_ridges(image, sigma=4.0)
plot(minima, figsize=(10, 10), display_range=True)
minima_thresholded = minima > 0.002
plot(minima_thresholded, figsize=(10, 10))
plot(maxima, figsize=(10, 10))

image -= np.min(image)
image /= np.max(image)
plot(minimum(image, footprint=disk(5)), figsize=(10, 10))
# plot(canny(gaussian(image, sigma=3), sigma=3), figsize=(15, 15))

# image = gaussian(image, sigma=3)
# # Compute the gradient using a 5x5 Sobel kernel
# sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=11)
# sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=11)

# plot(image)
# plot(sobel_x, figsize=(15, 15))
# plot(sobel_y, figsize=(15, 15))
# total_sobel = abs(sobel_x) + abs(sobel_y)
# plot(total_sobel, figsize=(15, 15))
# print(f"range: {np.min(total_sobel), np.max(total_sobel)}")
# plot(total_sobel < 100000)
# plot(laplace(image))

# footprint = disk(3)
# # footprint = np.ones((21, 21))

# filtered = sobel(image)
# filtered = np.max(filtered) - filtered
# print(f"filtered range: {np.min(filtered), np.max(filtered)}")
# plot(filtered, figsize=(15, 15), title="filtered")
# plot(filtered > 0.0, figsize=(15, 15))

# image = image - np.min(image)
# image = image / np.max(image)

# plot(image)
# filtered = minimum(image, footprint=disk(5))
# plot(filtered, figsize=(15, 15), title="filtered")

# _, ridges = detect_ridges(filtered, sigma=1.0)
# plot(ridges, figsize=(15, 15))

### Plot gallery

In [ ]:
fig, ax = plt.subplots(len(images), 2, figsize=(20, 300))
for index, (image, mask) in enumerate(zip(images, masks)):
    ax[index, 0].imshow(image)
    ax[index, 1].imshow(image > 2.3)

fig, ax = plt.subplots(1, 2, figsize=(15, 30))
ax[0].imshow(images[2])
ax[1].imshow(masks[2])
plt.show()

image = images[2]
mask = masks[2]

### Zoom in on one image

In [ ]:
image = images[2]
mask = masks[2]

# Zoom in on image
image = image[0:512, 0:512]
mask = mask[0:512, 0:512]

# Get a mask
labelled = label(mask)
plot(labelled, title="labelled")

# Get an individual mask
molecule_mask = labelled == 5
plot(molecule_mask, title="molecule mask")

# Get molecule heights
molecule = np.full(image.shape, 0.0)
molecule[molecule_mask] = image[molecule_mask]

plot(molecule)

In [ ]:
def dfs(array: np.ndarray, start: tuple, threshold: float):
    stack = [start]
    reachable_points = []
    visited = np.zeros_like(array, dtype=bool)
    while stack:
        point = stack.pop()
        if not visited[point]:
            visited[point] = True
            if array[point] > threshold:
                reachable_points.append(point)
                for neighbour in get_neighbours(point, array.shape):
                    if not visited[neighbour]:
                        stack.append(neighbour)
    return np.array(reachable_points)


threshold = 2.4

visited_points = np.zeros_like(molecule)

# Pick the highest point
highest_point_pos = np.unravel_index(np.argmax(molecule), molecule.shape)
highest_point_value = molecule[highest_point_pos]

print(f"molecule height range: {np.min(molecule), np.max(molecule)}")


current_point = highest_point_pos
for i in range(2920):
    neighbours = get_neighbours(current_point, molecule.shape)

    max_neighbour_height = 0
    max_neighbour_index = 0
    for neighbour_index, neighbour in enumerate(neighbours):
        neighbour_height = molecule[neighbour[0], neighbour[1]]
        if neighbour_height > max_neighbour_height and visited_points[neighbour[0], neighbour[1]] == 0:
            max_neighbour_height = neighbour_height
            max_neighbour_index = neighbour_index

    current_point = neighbours[max_neighbour_index]
    visited_points[current_point[0], current_point[1]] = 1


plot(visited_points)
plot(molecule)

# reachable_points = dfs(molecule, highest_point_pos, threshold=threshold)
# display_image = molecule.copy()
# display_image[highest_point_pos] = 10.0
# display_image[reachable_points[:, 0], reachable_points[:, 1]] = 10.0
# fig, ax = plt.subplots(figsize=(10, 10))
# ax.imshow(display_image)
# plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 10))
vmin, vmax = (np.min(image), np.max(image))
ax[0].imshow(image, vmin=vmin, vmax=vmax)
ax[1].imshow(mask)
ax[2].imshow(molecule, vmin=vmin, vmax=vmax)
plt.show()

# Pick the highest point
highest_point_pos = np.unravel_index(np.argmax(molecule), molecule.shape)
highest_point_value = molecule[highest_point_pos]
print(f"highest point value: {highest_point_value} position: {highest_point_pos}")
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(image)
ax.annotate("highest point", xy=(highest_point_pos[1], highest_point_pos[0]))
ax.scatter(highest_point_pos[1], highest_point_pos[0], c="red", marker=".")
plt.show()

# adaptive_threshold = highest_point_value
# adaptive_threshold_limit = np.median(molecule)
# adaptive_threshold_step = 0.5

# found_pixels = [highest_point_pos]

# found_pixels_image = np.zeros(image.shape).astype(bool)
# found_pixels_image[highest_point_pos] = True

# while adaptive_threshold > adaptive_threshold_limit:
#     print(f"threshold: {adaptive_threshold}")
#     # Get the points adjacent to the points in the found_pixels list that are not already in the found_pixels list and if they are above the threshold, add the point to the found_pixels list

#     # Get the found points from the binary image
#     found_points = np.argwhere(found_pixels_image == True)

#     # Find points next to the found points that exceed the threshold
#     for point in found_points:
#         y, x = point
#         for j in range(-1, 2):
#             for i in range(-1, 2):
#                 neighbour_y, neighbour_x = (y + j, x + i)
#                 if neighbour_y >= 0 and neighbour_y < image.shape[0] and neighbour_x >= 0 and neighbour_x < image.shape[1]:
#                     if image[neighbour_y, neighbour_x] > adaptive_threshold:
#                         found_pixels_image[neighbour_y, neighbour_x] = True

#     plt.imshow(found_pixels_image)
#     plt.show()

#     adaptive_threshold -= adaptive_threshold_step


threshold = 2.3
reachable_points = dfs(array=image, start=highest_point_pos, threshold=threshold)

display_image = image.copy()
display_image[highest_point_pos] = 10.0
display_image[reachable_points[:, 0], reachable_points[:, 1]] = 10.0
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(display_image)
plt.show()

reachable_points_image = np.zeros_like(image, dtype=bool)
reachable_points_image[reachable_points[:, 0], reachable_points[:, 1]] = True
plot(reachable_points_image, title="reachable points", figsize=(10, 10))
dists = distance_transform_edt(reachable_points_image)
plot(dists, title="dists", figsize=(20, 20))
plot(dists > 1, title="thresholded dists", figsize=(20, 20))

zeros_in_range = calc_zeros_in_range(array=reachable_points_image, range=3)
zeros_in_range[np.invert(reachable_points_image)] = 0
zeros_in_range[zeros_in_range <= 3] = 0
print(f"unique(zeros_in_range): {np.unique(zeros_in_range)}")

filtered = filter_by_percentile(image, percentile=40, kernel_size=21)
plot(filtered, title="local maxima (>40th percentile)", figsize=(10, 10))

fig, ax = plt.subplots(2, 2, figsize=(20, 20))
ax[0, 0].imshow(image)
ax[0, 0].set_title("molecule image")
ax[0, 1].imshow(reachable_points_image)
ax[0, 1].set_title("depth first search result")
ax[1, 0].imshow(dists > 1)
ax[1, 0].set_title("dfs - points within 1 pixels of a zero")
ax[1, 1].imshow(reachable_points_image - zeros_in_range.astype(bool).astype(int))
ax[1, 1].set_title("dfs - points within 3 pixels of 2 or more 0s")

In [ ]:
test = dists.copy()
plot(test)
# plot(test, figsize=(20, 20))


def calc_zeros_in_range(array: np.ndarray, range: int):
    kernel = np.ones((range, range))

    array_int = array.astype(int)

    array_inverted = 1 - array_int

    result = convolve(array_inverted, kernel, mode="constant", cval=0.0)

    return result


# # Find all the valss
# vals = test > 5
# inverted_vals = 1 - vals.astype(int)
# distance_to_five = distance_transform_edt(inverted_vals)
# plot(distance_to_five, title="distance_to_five", figsize=(20, 20))

vals = np.logical_or(test == 3, test == 4).astype(int)
vals += (test == 5).astype(int) * 2

inverted_vals = 1 - vals.astype(int)
distance_to_three = distance_transform_edt(inverted_vals)
plot(distance_to_three, title="distance_to_three", figsize=(20, 20))

plot(vals, figsize=(20, 20))


# far_from_5 = distance_to_five > 3
# plot(far_from_5)
# mol_far_from_5 = np.zeros_like(test)
# for j in range(test.shape[0]):
#     for i in range(test.shape[1]):
#         if far_from_5[j, i]:
#             mol_far_from_5[j, i] = test[j, i]
# plot(mol_far_from_5, figsize=(20, 20))


# # Find all the non-zero values that are not within x of a 5
# def calc_vals_in_range(array: np.ndarray, range: int, val: int):
#     all_vals_bool = array == val
#     plot(all_vals_bool)

#     kernel = np.ones((range, range))
#     array_int = all_vals_bool.astype(int)
#     result = convolve(array_int, kernel, mode="constant", cval=0.0)
#     return result

# vals_in_range = calc_vals_in_range(test, range=5, val=5)
# plot(vals_in_range, figsize=(20, 20))

# Select 3s not within 3 of a six and scale up
# plot(vals, figsize=(20, 20))

In [ ]:
def filter_by_percentile(arr, percentile: float, kernel_size=3):
    # Define the function to apply to each window
    index_of_middle = np.square(kernel_size) // 2

    def func(window):
        # Find the median value of the window
        threshold = np.percentile(window, percentile)
        # Set the center pixel to 0 if it is less than the median
        if window[index_of_middle] < threshold:
            return 0
        else:
            return window[index_of_middle]

    # Create a new array by applying the function to each 3x3 window
    result = generic_filter(arr, func, size=kernel_size, mode="constant", cval=0.0)

    return result

In [ ]:
sub_image = image[200:380, 280:400]
# sub_image = image[250:300, 300:400]
plot(sub_image)
filtered = filter_by_percentile(sub_image, percentile=40, kernel_size=23)
plot(filtered)

In [ ]:
threshold = 2

for image in images:
    mask = image > threshold

    fig, ax = plt.subplots(1, 3, figsize=(30, 10))
    ax[0].imshow(image)
    ax[1].imshow(mask)